In [10]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.13.0'

In [11]:
# Input data
df = pd.read_csv('../input_data/CSVs/superconductivity.csv')
# Drop columns with null entries
df.dropna(inplace=True)
df

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.00
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.00
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.00
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.00
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21258,4,106.957877,53.095769,82.515384,43.135565,1.177145,1.254119,146.88130,15.504479,65.764081,...,3.555556,3.223710,3.519911,1.377820,0.913658,1,2.168889,0.433013,0.496904,2.44
21259,5,92.266740,49.021367,64.812662,32.867748,1.323287,1.571630,188.38390,7.353333,69.232655,...,2.047619,2.168944,2.038991,1.594167,1.337246,1,0.904762,0.400000,0.212959,122.10
21260,2,99.663190,95.609104,99.433882,95.464320,0.690847,0.530198,13.51362,53.041104,6.756810,...,4.800000,4.472136,4.781762,0.686962,0.450561,1,3.200000,0.500000,0.400000,1.98
21261,2,99.663190,97.095602,99.433882,96.901083,0.690847,0.640883,13.51362,31.115202,6.756810,...,4.690000,4.472136,4.665819,0.686962,0.577601,1,2.210000,0.500000,0.462493,1.84


In [12]:
# Set target to be critical tempurature, with feature set being all other data
target = df[['critical_temp']]
features = df.drop(['critical_temp'], axis=1)

In [13]:
from sklearn.preprocessing import StandardScaler
# Use StandardScaler to rescale the data around 0 and scale based on unit variance
scaler = StandardScaler()
feature_array = scaler.fit_transform(features)
features = pd.DataFrame(feature_array, columns=features.columns)

In [14]:
# Create a model to train on the dataframe
model = tf.keras.Sequential()

# Input layer with shape matching the feature set
model.add(tf.keras.layers.InputLayer(input_shape=(features.shape[1],), name='Input_layer'))

# 4 Hidden layers with 64, 32, 16 and 8 nodes per layer
model.add(tf.keras.layers.Dense(64, name='Dense_layer_1', activation='relu')) # Use ReLU activation function for this layer
model.add(tf.keras.layers.Dense(32, name='Dense_layer_2'))
model.add(tf.keras.layers.Dense(16, name='Dense_layer_3'))
model.add(tf.keras.layers.Dense(8, name='Dense_layer_4'))

# Output layer with connections to all previous nodes
model.add(tf.keras.layers.Dense(1, name='Output_layer'))

In [15]:
# Use the RMSprop optimiser function and mean squared error for loss
model.compile(tf.optimizers.RMSprop(0.001), loss='mse')
# Add a tensorboard callback to create logs for future reference
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

In [18]:
# Train the model over 100 epochs and add a 20% validation split to the input data for evaluation after training
model.fit(x=features.to_numpy(), y=target.to_numpy(), 
          epochs=100, callbacks=[tensorboard_callback], batch_size=32, validation_split=0.2)

Epoch 1/100
532/532 [==============================] - 1s 3ms/step - loss: 160.2928 - val_loss: 268.3968
Epoch 2/100
532/532 [==============================] - 2s 3ms/step - loss: 159.5576 - val_loss: 283.8095
Epoch 3/100
532/532 [==============================] - 3s 5ms/step - loss: 160.3793 - val_loss: 266.1397
Epoch 4/100
532/532 [==============================] - 2s 5ms/step - loss: 159.9051 - val_loss: 273.5670
Epoch 5/100
532/532 [==============================] - 3s 5ms/step - loss: 157.4365 - val_loss: 270.1654
Epoch 6/100
532/532 [==============================] - 2s 5ms/step - loss: 159.2270 - val_loss: 288.4320
Epoch 7/100
532/532 [==============================] - 2s 4ms/step - loss: 159.5367 - val_loss: 287.6465
Epoch 8/100
532/532 [==============================] - 2s 4ms/step - loss: 158.6662 - val_loss: 281.0269
Epoch 9/100
532/532 [==============================] - 3s 5ms/step - loss: 156.8927 - val_loss: 270.2240
Epoch 10/100
532/532 [==============================] -

In [19]:
# Evaluate the model on the 20% remaining data
loss = model.evaluate(features.to_numpy(), target.to_numpy())
print('loss:', loss)

665/665 [==============================] - 3s 4ms/step - loss: 174.4449
loss: 174.4449462890625
